In [ ]:
import os
import sys

sys.path.append(os.path.abspath("src/"))

import logging
import sys

import hydra
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import torch
import yaml
from hydra import initialize_config_dir
from joblib import Memory
from omegaconf import DictConfig, OmegaConf

from evaluators import Evaluator
from models import FastTextWrapper
from src.datasets import SoftClassifDataset, TextClassificationDataModule
from utils.data import PATHS, get_df_naf, get_file_system, get_processed_data, get_test_data, get_Y
from utils.evaluation import (
    get_fasttext_preds,
    get_ground_truth,
    get_label_mapping,
    sort_and_get_pred,
)
from utils.mappings import mappings
from utils.mlflow import create_or_restore_experiment
from utils.validation_viz import (
    calibration_curve,
    confidence_histogram,
    get_automatic_accuracy,
)

%load_ext autoreload
%autoreload 2

In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = "https://projet-ape-mlflow.user.lab.sspcloud.fr/" 
# model_name = "FastText-pytorch"
# module = mlflow.pytorch.load_model(f"models:/{model_name}/latest")

run_id = "01fd012d1d8f45828a889efd8cb926ec"
logged_model = f'runs:/{run_id}/model'
# Load model as a PyFuncModel.
module = mlflow.pytorch.load_model(logged_model, map_location=torch.device('cpu'))

In [ ]:
# Download the artifact directory (e.g., to a temp dir)
local_artifacts_path = mlflow.artifacts.download_artifacts(run_id=run_id)

# Load the YAML config
with open(f"{local_artifacts_path}/hydra_config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
cfg: DictConfig = OmegaConf.create(config)

In [ ]:
trainer = hydra.utils.instantiate(cfg.model.trainer)

In [ ]:
datamodule = TextClassificationDataModule(
                    cfg.data,
                    cfg.tokenizer,
                    cfg.dataset,
                    batch_size = 256)

In [ ]:
datamodule.setup()

In [ ]:
val_loader = datamodule.val_dataloader()
batch = next(iter(val_loader))

In [ ]:
predictions = trainer.predict(module, val_loader)
predictions_tensor = torch.cat(predictions).cpu()

In [ ]:
df = datamodule.df_val
Y = datamodule.Y
df_naf = get_df_naf(revision=cfg.data.revision)
true_values = get_ground_truth(df, Y)
sorted_confidence, predicted_confidence, predicted_class = sort_and_get_pred(predictions=predictions_tensor, true_values=true_values)
naf_predictions = get_pred_nafs(predicted_class, revision=cfg.data.revision)
all_level_preds = get_all_levels(naf_predictions, df_naf)
all_level_ground_truth = get_all_levels(df, df_naf, col=Y, revision=cfg.data.revision)

In [ ]:
(all_level_preds == all_level_ground_truth).mean(axis=0)

In [ ]:
naf_predictions.merge(df_naf, left_on="APE_NIV5_pred", right_on="APE_NIV5", how="left")

In [ ]:
fasttext_preds_labels, fasttext_preds_scores = get_fasttext_preds(revision=cfg.data.revision)

In [ ]:
mapping = get_label_mapping(cfg.data.revision)
fasttext_preds_labels_int = fasttext_preds_labels.map(mapping.get)

In [ ]:
true_values = get_ground_truth(datamodule.df_test, datamodule.Y)
thresholds = np.linspace(0, 1, 100)
ft_plot = get_automatic_accuracy(
    thresholds,
    np.clip(fasttext_preds_scores.values.reshape(-1), 0, 1),
    fasttext_preds_labels_int.values.reshape(-1),
    true_values,
)
mask_ft = ft_plot[0] > 0

# Create the matplotlib figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Plot ft data
ax.scatter(ft_plot[0][mask_ft], ft_plot[1][mask_ft], label="ft")

# Set labels and title
ax.set_xlabel("Pourcentage de codif automatique")
ax.set_ylabel("Accuracy")

# Add legend
ax.legend(loc="upper right", fancybox=True, shadow=True)

# Set grid
ax.grid(True, linestyle="--", alpha=0.7)

# Adjust layout
plt.tight_layout()

In [ ]:
targets.max(dim=1).values

In [ ]:
train_loader = datamodule.train_dataloader()
batch = next(iter(train_loader))
inputs, targets = batch[:-1], batch[-1]
outputs = module.forward(inputs)
loss = module.loss(outputs, targets)
accuracy = module.accuracy_fn(outputs, targets.argmax(dim=1))
loss, accuracy

In [ ]:
batch = next(iter(train_loader))
inputs, targets = batch[:-1], batch[-1]
targets = targets / targets.sum(dim=1, keepdim=True)
targets[0].max()

In [ ]:
target

In [ ]:
text, categorical_variables = (
            df_test[text_feature].values,
            df_test[categorical_features].values,
        )

dataset = SoftClassifDataset(
    texts=text,
    categorical_variables=categorical_variables,
    tokenizer=module.model.tokenizer,
    outputs=df_test[Y].values,
    similarity_coefficients=[0.01, 0.1, 0.1, 0.1, 0.5],
    revision=cfg_dict["data"]["revision"],
)
dataloader = dataset.create_dataloader(
    batch_size=10, shuffle=False, num_workers=12
)

In [ ]:

df_train = df_train.sample(frac=0.001)
df_val = df_val.sample(frac=0.01)
df_test = df_test.sample(frac=0.01)

train_text, train_categorical_variables = (
            df_train[cfg_dict["data"]["text_feature"]].values,
            df_train[cfg_dict["data"]["categorical_features"]].values,
        )
val_text, val_categorical_variables = (
    df_val[cfg_dict["data"]["text_feature"]].values,
    df_val[cfg_dict["data"]["categorical_features"]].values,
)
test_text, test_categorical_variables = (
    df_test[cfg_dict["data"]["text_feature"]].values,
    df_test[cfg_dict["data"]["categorical_features"]].values,
)

In [ ]:

tokenizer = TOKENIZERS[cfg_dict["tokenizer"]["tokenizer_name"]](
            **cfg_dict["tokenizer"], training_text=train_text
        )

num_rows = tokenizer.num_tokens + tokenizer.get_nwords() + 1
padding_idx = num_rows - 1
num_classes = max(mappings[Y].values()) + 1
categorical_vocab_sizes = []
for feature in cfg_dict["data"]["categorical_features"]:
    if feature == "SRF":
        categorical_vocab_sizes.append(5)
    else:
        categorical_vocab_sizes.append(max(mappings[feature].values()) + 1)

model = MODELS[cfg_dict["model"]["model_name"]](
    **cfg_dict["model"]["model_params"],
    tokenizer=tokenizer,
    num_rows=num_rows,
    num_classes=num_classes,
    categorical_vocabulary_sizes=categorical_vocab_sizes,
    padding_idx=padding_idx,
)

loss = LOSSES[cfg_dict["model"]["train_params"]["loss_name"]]()
optimizer = OPTIMIZERS[
    cfg_dict["model"]["train_params"]["optimizer_name"]
]  # without the () !
scheduler = SCHEDULERS[cfg_dict["model"]["train_params"]["scheduler_name"]]

module = MODULES[cfg_dict["model"]["model_name"]](
    model=model,
    loss=loss,
    optimizer=optimizer,
    scheduler=scheduler,
    **cfg_dict["model"]["train_params"],
)

In [ ]:
trainer = TRAINERS[cfg_dict["model"]["train_params"]["trainer_name"]](
            **cfg_dict["model"]["train_params"],
        )


In [ ]:
dataset_class = SoftClassifDataset


train_dataset = dataset_class(
    texts=train_text,
    categorical_variables=train_categorical_variables,
    tokenizer=tokenizer,
    outputs=df_train[Y].values,
    revision=cfg_dict["data"]["revision"],
    similarity_coefficients=similarity_coefficients,
)
val_dataset = dataset_class(
    texts=val_text,
    categorical_variables=val_categorical_variables,
    tokenizer=tokenizer,
    outputs=df_val[Y].values,
    revision=cfg_dict["data"]["revision"],
    similarity_coefficients=similarity_coefficients,
)

test_dataset = dataset_class(
    texts=test_text,
    categorical_variables=test_categorical_variables,
    tokenizer=tokenizer,
    outputs=df_test[Y].values,
    revision=cfg_dict["data"]["revision"],
    similarity_coefficients=similarity_coefficients,
)

In [ ]:
train_dataloader = train_dataset.create_dataloader(
    **cfg_dict["model"]["train_params"]
)
val_dataloader = val_dataset.create_dataloader(**cfg_dict["model"]["train_params"])
test_dataloader = test_dataset.create_dataloader(
    **cfg_dict["model"]["train_params"]
)

In [ ]:
def run_eval(df, dataloader, suffix='val'):
    """
    Run evaluation on the given dataloader and log the results.
    """

    predictions = trainer.predict(module, dataloader) # accumulates predictions over batches
    predictions_tensor = torch.cat(predictions).cpu().numpy() # (num_test_samples, num_classes)

    # Use your aggregation function
    aggregated_results = Evaluator.get_aggregated_preds(
        df=df,
        Y=Y,
        predictions=predictions_tensor,
        top_k=1
    )

    display(aggregated_results)

    accuracy = Evaluator.compute_accuracies(aggregated_preds=aggregated_results, suffix=suffix)

    return aggregated_results

run_eval(df_val, val_dataloader, suffix='val')
run_eval(df_test, test_dataloader, suffix='test')